# Particle Life

In [ ]:
import numpy as np
from ipycanvas import Canvas, hold_canvas
from IPython.display import display
import numpy as np
from utils import visually_distinct_colors
import time

n_types = 3
n_particles = 1000
shape = [500, 500]
max_distance = 30.0
damping = 0.5
step_size = 0.3
# seed
np.random.seed(1)
interaction_matrix = np.random.randn(n_types, n_types)-0.5
particles_type = np.random.randint(0, n_types, n_particles)
particle_pos = np.random.rand(n_particles, 2) * shape
particle_vel = (np.random.rand(n_particles, 2) - 0.5)
particle_force = np.zeros((n_particles, 2))
colors = visually_distinct_colors(n_types, seed=42)*255


# pre-compute nxn matrix of interaction strengths
interaction_strengths = np.zeros((n_particles, n_particles))
for i in range(n_types):
    for j in range(n_types):
        mask_i = (particles_type == i)
        mask_j = (particles_type == j)
        interaction_strengths[np.ix_(mask_i, mask_j)] = interaction_matrix[i, j]


canvas = Canvas(width=shape[0], height=shape[1])
display(canvas)

def vec_distance_func(dist, interaction_strength, max_distance, beta=0.3):
    r = dist / max_distance
    
    # Repulsion when r < beta
    result = np.where(r < beta, r / beta - 1.0, 0.0)
    
    # Attraction/repulsion when beta < r < 1
    mask = (beta < r) & (r < 1.0)
    result = np.where(mask, 
                      interaction_strength * (1.0 - np.abs(2*r - 1 - beta) / (1.0 - beta)),
                      result)
    
    # Zero force when r >= 1
    return result


def display(particle_pos, particle_type, n_types, colors):
    with hold_canvas(canvas):
        canvas.fill_style = 'black'
        canvas.fill_rect(0, 0, shape[0], shape[1])
        colors = np.take(colors, particle_type, axis=0)
        canvas.fill_styled_circles(
            particle_pos[:, 0],
            particle_pos[:, 1],
            radius=3,
            color=colors,
        )


def update_forces():
    global particle_pos, particle_vel, particle_force, max_distance
    # Reset forces
    particle_force[:] = 0

    # Compute all pairs of distances
    diff = particle_pos[np.newaxis, :, :] - particle_pos[:, np.newaxis, :]
    
    # Wrap around boundaries for distance calculation
    diff = np.where(diff > shape[0] / 2, diff - shape[0], diff)
    diff = np.where(diff < -shape[0] / 2, diff + shape[0], diff)
    
    dist = np.linalg.norm(diff, axis=-1) + 1e-6  # Avoid division by zero

    # Only compute forces for particles within max_distance
    mask = dist < max_distance
    
    # Apply vec_distance_func only on masked elements
    force_magnitude = np.zeros_like(dist)
    force_magnitude[mask] = vec_distance_func(dist[mask], interaction_strengths[mask], max_distance)

    # Normalize direction
    direction = diff / dist[..., np.newaxis]    
    # Accumulate forces
    force = (force_magnitude[..., np.newaxis] * direction).sum(axis=1)
    particle_force += force

def apply_forces():
    global particle_pos, particle_vel, particle_force, damping, step_size

    # Update velocities and positions

    particle_vel *= damping
    particle_vel += particle_force * step_size
    particle_pos += particle_vel * step_size

    # wrap around boundaries
    particle_pos[:, 0] = particle_pos[:, 0] % shape[0]
    particle_pos[:, 1] = particle_pos[:, 1] % shape[1]
 

acc_t_update = 0   
for i in range(1000):
    t0 = time.time()
    update_forces()
    acc_t_update += time.time() - t0
    apply_forces()

    display(particle_pos, particles_type, n_types, colors)
    # draw text on canvas
    canvas.fill_style = 'white'
    canvas.font = '16px sans-serif'
    canvas.fill_text(f'Frame: {i}, t_update: {acc_t_update/(i+1):.4f}s', 10, 20)



In [ ]:
particle_pos